In [1]:
#!sudo /opt/conda/bin/conda install conda-build -y

In [2]:
#!sudo /opt/conda/bin/conda-develop -n QML-QPF PATH /workspaces/QML-QPF/mosaiQue

In [ ]:
import mlcroissant as mlc
import pandas as pd

# Fetch the Croissant JSON-LD
croissant_dataset = mlc.Dataset('www.kaggle.com/datasets/surajghuwalewala/ham1000-segmentation-and-classification/croissant/download')

# Check what record sets are in the dataset
record_sets = croissant_dataset.metadata.record_sets
print(record_sets)

# Fetch the records and put them in a DataFrame
record_set_df = pd.DataFrame(croissant_dataset.records(record_set=record_sets[0].uuid))
record_set_df.head()


In [3]:

#import tensorflow as tf
#from tensorflow import keras
import mosaique as mq
from concurrent.futures import ProcessPoolExecutor, as_completed
import itertools
import numpy as np
import pennylane as qml
import os
import time
from mosaique.models.operation import OperationLayer
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'


def operation():
    dev = qml.device("default.qubit.tf", wires=4)
    @qml.qnode(dev, interface='tf')
    def cnot(inputs):
        inputs = inputs * np.pi
        qml.AngleEmbedding(inputs[:,...], wires=range(4), rotation='Y')

        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[2, 3])

        # Measurement producing 4 classical output values
        return [qml.expval(qml.PauliZ(j)) for j in range(4)]
    return cnot

2024-11-25 13:50:57.540156: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-25 13:50:58.265543: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
def color_merge(dataset):
    # Convert RGB to grayscale using the luminance formula
    luminance = 0.299 * dataset[:,:,:,0] + 0.587 * dataset[:,:,:,1] + 0.114 * dataset[:,:,:,2]
    # Average the RGB channels
    average = np.mean(dataset, axis=-1)
    # Combine luminance and average using a weighted sum (you can adjust the weights)
    return 0.7 * luminance + 0.3 * average

def preset():
    from tensorflow import keras
    cifar_dataset = keras.datasets.cifar10
    tr_layer = mq.ConvolutionLayer4x4("cifar_train")
    te_layer = mq.ConvolutionLayer4x4("cifar_test")
    (tr_images, tr_labels), (te_images, te_labels) = cifar_dataset.load_data()
    tr_images = color_merge(tr_images)
    te_images = color_merge(te_images)
    tr_layer.fit(tr_images)
    te_layer.fit(te_images)
    tr_images = tr_layer.transform(tr_images)
    te_images = te_layer.transform(te_images)
    return ((tr_layer, (tr_images, tr_labels)), (te_layer,(te_images, te_labels)))

with ProcessPoolExecutor() as executor:
    future = executor.submit(preset)

((train_layer, (train_images, train_labels)), (test_layer,(test_images, test_labels))) = future.result() #This blocks until the task completes

In [5]:
permutations = np.asarray(list(itertools.permutations(range(4))))

def pool(x, call, p, l):
    #sleep_duration = np.random.randint(1, 5) # Sleep between 1 and 4 seconds
    #time.sleep(sleep_duration)
    op = OperationLayer(call())
    predict = l.post_transform(op.pre_op.predict(x,batch_size=1000))
    l.save(predict, p)



In [6]:
#[pool(x=train_images[:,:,p], call=operation, p=p, l=train_layer) for p in permutations[:1]]


In [7]:
 
for j in range(3):
    with ProcessPoolExecutor(8) as executor:
        runner = {
            executor.submit(pool, x=train_images[:,:,p], call=operation, p=p, l=train_layer): p for p in permutations[8*j:8*(j+1)]
        }
        for future in as_completed(runner):
            runner.pop(future)
#3 min 43 sec

2024-11-25 13:51:05.677923: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:238] Using CUDA malloc Async allocator for GPU: 0
2024-11-25 13:51:05.678238: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6795 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:b3:00.0, compute capability: 7.5
2024-11-25 13:51:06.845591: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:238] Using CUDA malloc Async allocator for GPU: 0
2024-11-25 13:51:06.845894: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6461 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:b3:00.0, compute capability: 7.5
I0000 00:00:1732542666.954224  646899 service.cc:145] XLA service 0x71d9ac002700 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732542666.954302  6468

 3/50 ━━━━━━━━━━━━━━━━━━━━ 9s 192ms/step

I0000 00:00:1732542667.932725  646899 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
I0000 00:00:1732542668.031652  646955 service.cc:145] XLA service 0x71d9ac002700 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732542668.031686  646955 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2080, Compute Capability 7.5
2024-11-25 13:51:08.055965: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:238] Using CUDA malloc Async allocator for GPU: 0
2024-11-25 13:51:08.056283: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5817 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:b3:00.0, compute capability: 7.5
2024-11-25 13:51:08.058869: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_

 1/50 ━━━━━━━━━━━━━━━━━━━━ 1:49 2s/step 275ms/step

2024-11-25 13:51:09.291858: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:238] Using CUDA malloc Async allocator for GPU: 0
2024-11-25 13:51:09.292162: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5173 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:b3:00.0, compute capability: 7.5
I0000 00:00:1732542669.309849  647048 service.cc:145] XLA service 0x71d9a8023850 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732542669.309936  647048 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2080, Compute Capability 7.5
2024-11-25 13:51:09.350946: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-25 13:51:09.442611: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


...

12/50 ━━━━━━━━━━━━━━━━━━━━ 34s 920ms/step

I0000 00:00:1732542847.079728  651256 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 1/50 ━━━━━━━━━━━━━━━━━━━━ 4:58 6s/step 963ms/step

I0000 00:00:1732542848.665151  651357 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


50/50 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step
50/50 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step
50/50 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step
50/50 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step
50/50 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step
50/50 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step
50/50 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step
50/50 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step


In [8]:


for j in range(3):
    with ProcessPoolExecutor(8) as executor:
        runner = {
            executor.submit(pool, x=test_images[:,:,p], call=operation, p=p, l=test_layer): p for p in permutations[8*j:8*(j+1)]
        }
        for future in as_completed(runner):
            runner.pop(future)
#50 sec

2024-11-25 13:55:20.291389: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:238] Using CUDA malloc Async allocator for GPU: 0
2024-11-25 13:55:20.291705: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6795 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:b3:00.0, compute capability: 7.5
2024-11-25 13:55:20.554025: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:238] Using CUDA malloc Async allocator for GPU: 0
2024-11-25 13:55:20.554331: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6621 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:b3:00.0, compute capability: 7.5
2024-11-25 13:55:20.797371: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:238] Using CUDA malloc Async allocator for GPU: 0
2024-11-25 13:55:20.797722: I tensorflow/core/common_runtime/gpu/gpu

 1/10 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step

2024-11-25 13:55:22.378878: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:238] Using CUDA malloc Async allocator for GPU: 0
2024-11-25 13:55:22.379366: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4643 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:b3:00.0, compute capability: 7.5
I0000 00:00:1732542922.468033  652688 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
I0000 00:00:1732542922.579520  652879 service.cc:145] XLA service 0x71d9cc024080 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732542922.579560  652879 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2080, Compute Capability 7.5
2024-11-25 13:55:22.616243: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_

 2/10 ━━━━━━━━━━━━━━━━━━━━ 5s 746ms/step

I0000 00:00:1732542923.180998  653057 service.cc:145] XLA service 0x71d9d8002350 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732542923.181035  653057 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2080, Compute Capability 7.5
2024-11-25 13:55:23.215944: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-25 13:55:23.320719: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1732542923.338243  653133 service.cc:145] XLA service 0x71d9cc002700 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732542923.338276  653133 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2080, Compute Capability 7.5
2024-11-25 13:55:23.369451: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] dis

...

 5/10 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step

I0000 00:00:1732542973.252110  654930 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 2/10 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step

I0000 00:00:1732542973.475763  655013 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


10/10 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step
